# Résolution d'une équation aux dérivées partielles (e.d.p.) avec la méthode des différences finies 

####Mercredi 25 Mars 2015
#####Claire Parquier (parqui_c)

Le problème à résoudre est l'équation de la chaleur sur le domaine $\Omega$ :

$$-k \Delta T(x, y) = f(x, y) ~~~~~~~~ (x, y) \in \Omega \subset R^2$$

où :

* x et y sont les variables d'espace,
* $T$ est l'inconnue, la température à calculer,
* $k$ est la conductivité du matériau (connue),
* $f(x, y)$ représente les sources de chaleur (connues).

On utilise les bibliothèques suivantes :

In [214]:
from matplotlib import pyplot
from scipy import linalg
import numpy

## 1 - Maillage

On réutilise le namedtuple Node pour représenter un point :

In [215]:
from collections import namedtuple

Node = namedtuple('Node', ['id', 'x', 'y', 'bc', 'bv', 'left', 'right', 'up', 'down'])

On indique d'abord le bord (conditions à la limite), en précisant les x et y sur lesquels se trouvent les coins :

In [216]:
# p2-------p3
#  | p7-p6  |
#  |  |  |  |
# p1-p8 p5-p4

lx1 = 0
lx2 = 2
lx3 = 8
lx4 = 10

ly1 = 0
ly2 = 5
ly3 = 6


p1 = (lx1, ly1)
p2 = (lx1, ly3)
p3 = (lx4, ly3)
p4 = (lx4, ly1)
p5 = (lx3, ly1)
p6 = (lx3, ly2)
p7 = (lx2, ly2)
p8 = (lx2, ly1)

# shape with values on border :
boundary_conditions = [
    [p1, p2, 1],
    [p2, p3, 0.2],
    [p3, p4, 0.1], 
    [p4, p5, 0.5], 
    [p5, p6, 0.1],  
    [p6, p7, 0.4],  
    [p7, p8, 1], 
    [p8, p1, 1], 
    ]

"""
# another example with a T shape

# p2-------p3
#  |        |
# p1-p8 p5-p4
#     |  |
#    p7-p6

lx1 = 0
lx2 = 4
lx3 = 6
lx4 = 10

ly1 = 0
ly2 = 5
ly3 = 6


p1 = (lx1, ly2)
p2 = (lx1, ly3)
p3 = (lx4, ly3)
p4 = (lx4, ly2)
p5 = (lx3, ly2)
p6 = (lx3, ly1)
p7 = (lx2, ly1)
p8 = (lx2, ly2)

# shape with values on border :
boundary_conditions = [
    [p1, p2, 0],
    [p2, p3, 0.2],
    [p3, p4, 0.1], 
    [p4, p5, 0.5], 
    [p5, p6, 0.1],  
    [p6, p7, 0.4],  
    [p7, p8, 0], 
    [p8, p1, 0], 
    ]
"""

u'\n# another example with a T shape\n\n# p2-------p3\n#  |        |\n# p1-p8 p5-p4\n#     |  |\n#    p7-p6\n\nlx1 = 0\nlx2 = 4\nlx3 = 6\nlx4 = 10\n\nly1 = 0\nly2 = 5\nly3 = 6\n\n\np1 = (lx1, ly2)\np2 = (lx1, ly3)\np3 = (lx4, ly3)\np4 = (lx4, ly2)\np5 = (lx3, ly2)\np6 = (lx3, ly1)\np7 = (lx2, ly1)\np8 = (lx2, ly2)\n\n# shape with values on border :\nboundary_conditions = [\n    [p1, p2, 0],\n    [p2, p3, 0.2],\n    [p3, p4, 0.1], \n    [p4, p5, 0.5], \n    [p5, p6, 0.1],  \n    [p6, p7, 0.4],  \n    [p7, p8, 0], \n    [p8, p1, 0], \n    ]\n'

Puis la conductivité du milieu :

In [217]:
COND = 1

La source de chaleur :

In [218]:
heat = lambda x, y: (0, 5)[(1 < x < 2 and 0.2 < y < 1.2)
                           or (0 < x < 1 and 3 < y < 4)
                           or (8.5 < x < 9.5 and 0.1 < y < 0.8)]

On crée ensuite un maillage :

In [219]:
# carefull = all vertical and horizontal limits MUST be in the list
x_mesh = numpy.concatenate( [numpy.linspace(lx1,       lx1 + 0.3, 2, endpoint=False),
                             numpy.linspace(lx1 + 0.3, lx2 - 0.3, 6, endpoint=False),
                             numpy.linspace(lx2 - 0.3, lx2,       2, endpoint=False),
                             numpy.linspace(lx2,       lx3,       8, endpoint=False),
                             numpy.linspace(lx3,       lx3 + 0.3, 2, endpoint=False),
                             numpy.linspace(lx3 + 0.3, lx4 - 0.3, 6, endpoint=False),
                             numpy.linspace(lx4 - 0.3, lx4,       2)] )

y_mesh = numpy.concatenate( [numpy.linspace(ly1,       ly1 + 0.4, 10, endpoint=False),
                             numpy.linspace(ly1 + 0.4, ly2,       20, endpoint=False),
                             numpy.linspace(ly2,       ly2 + 0.2, 2,  endpoint=False),
                             numpy.linspace(ly2 + 0.2, ly3,       6)] )

On cherche maintenant boundary, la frontière, qu'on va calculer à partir de boundary_condition et des points du maillage :

In [220]:
import functools

boundary = {}
get_inside = lambda v0, v1: functools.partial(filter, lambda v: min(v0, v1) <= v <= max(v0, v1))

for ((x0, y0), (x1, y1), temperature) in boundary_conditions:
    if y0 == y1:
        for x in get_inside(x0, x1)(x_mesh):
            if (x, y0) in boundary:
                boundary[(x, y0)] = (boundary[(x, y0)] + temperature) / 2
            else:
                boundary[(x, y0)] = temperature
    elif x0 == x1:
        for y in get_inside(y0, y1)(y_mesh):
            if (x0, y) in boundary:
                boundary[(x0, y)] = (boundary[(x0, y)] + temperature) / 2
            else:
                boundary[(x0, y)] = temperature

Enfin, on construit le maillage avec tout ses noeuds.

In [221]:
plots = {}

on_boundary = lambda i, j: (x_mesh[i], y_mesh[j]) in boundary


for j, y in enumerate(y_mesh):
    in_mesh = False
    for i, x in enumerate(x_mesh):
        if on_boundary(i, j):
            plots[(i, j)] = Node((i, j), x, y, True, boundary[(x, y)], None, None, None, None)
            # we cross a wall
            if ((i <= 0 or not on_boundary(i - 1, j))
                and (i + 1 >= len(x_mesh) or not on_boundary(i + 1, j))):
                in_mesh = not in_mesh
            # we go along a wall
            elif (i + 1 < len(x_mesh) and (not on_boundary(i + 1, j))):
                # the node under the next one is not in the mesh
                if (not (i + 1, j - 1) in plots):
                    in_mesh = False
                # the node under the next one is in the mesh
                else:
                    in_mesh = not plots[(i + 1, j - 1)].bc
        elif in_mesh:
            plots[(i, j)] = Node((i, j), x, y, False, heat(x, y), (i - 1, j), (i + 1, j), (i, j + 1), (i, j - 1))

## 2 - Résolution de l'équation aux dérivées partielles

On utilise la méthode des différences finies :

$$-k \Delta T(x, y) = f(x, y) ~~~~~~~~ (x, y) \in \Omega \subset R^2$$

On peut approximer avec une erreur $\varepsilon$ grâce au théorème de Taylor :

$$ T(x + \varepsilon_1, y) = T(x, y) + \varepsilon_1 \frac{\partial T}{\partial x}(x, y) + \frac{\varepsilon_1^2}{2}\frac{\partial^2 T}{\partial x^2}(x, y) + O(\varepsilon_1^3)$$

$$ T(x - \varepsilon_2, y) = T(x, y) - \varepsilon_2 \frac{\partial T}{\partial x}(x, y) + \frac{\varepsilon_2^2}{2}\frac{\partial^2 T}{\partial x^2}(x, y) + O(\varepsilon_2^3)$$

On a donc :

$$ \varepsilon_2 T(x + \varepsilon_1, y) + \varepsilon_1 T(x - \varepsilon_2, y) = (\varepsilon_1 + \varepsilon_2) T(x, y) + \frac{\varepsilon_1^2 \varepsilon_2 + \varepsilon_2^2 \varepsilon_1}{2}\frac{\partial^2 T}{\partial x^2}(x, y) + O(\varepsilon_1^3 \varepsilon_2 + \varepsilon_2^3 \varepsilon_1)$$

On en déduit :

$$ \frac{\partial^2 T}{\partial x^2}(x, y) \approx \frac{2}{\varepsilon_1^2 \varepsilon_2 + \varepsilon_2^2 \varepsilon_1} (\varepsilon_2 T(x + \varepsilon_1, y) + \varepsilon_1 T(x - \varepsilon_2, y) - (\varepsilon_1 + \varepsilon_2) T(x, y))$$

On peut ainsi construire la matrice à résoudre :



In [222]:
needed = list(filter(lambda x: not x.bc, plots.values()))
rows = {t.id: i for i, t in enumerate(needed)}
n = len(needed)

# We want to solve Mat * X = B

Mat = numpy.zeros((n, n))
B = numpy.zeros(n)

f_error = lambda x, lhs, rhs: (-2 / (abs(x - lhs) * (x - rhs) ** 2 + abs(x - rhs) * (x - lhs) ** 2),
                               abs(x - rhs), abs(x - lhs))
row_of = lambda x: rows[x.id]

for p in needed:
    cr = row_of(p)
    B[cr] += p.bv / COND
    
    for sda, sdb, sd in (('left', 'right', 'x'), ('up', 'down', 'y')):
        da = plots[getattr(p, sda)]
        db = plots[getattr(p, sdb)]
        d, e1, e2 = f_error(getattr(p, sd), getattr(da, sd), getattr(db, sd))
        Mat[(cr, cr)] -= (e1 + e2) * d
        for sub, e in ((da, e1), (db, e2)):
            if sub.bc:
                B[cr] -= e * d * sub.bv
            else:
                if sd == 'x':
                    Mat[(cr, row_of(sub))] += e * d
                elif sd == 'y':
                    Mat[(cr, row_of(sub))] += e * d
                    

Notre matrice est maintenant remplie, on peut résoudre l'équation, et insérer les résultats dans la map values (point -> valeur) :

In [223]:
X = linalg.solve(Mat, B)
values = {p.id : X[row_of(p)] for p in needed}

Et ainsi créer la matrice résultat :

In [224]:
z = numpy.zeros((len(y_mesh), len(x_mesh)))

for i in range(len(x_mesh)):
    for j in range(len(y_mesh)):
        try:
            p = plots[(i, j)]
            z[(j, i)] = p.bv if p.bc else values[p.id]
        except KeyError:
            z[(j, i)] = numpy.nan

## 3 - Affichage

On peut maintenant afficher la simulation.

On commence d'abord par récupérer la liste des points en X et en Y.

zip(*l) correspond à la fonction inverse de zip(l).

In [225]:
x_plots, y_plots = zip(*boundary)

On affiche la simulation avec contourf et les points du maillage à la frontière avec scatter.

In [226]:
pyplot.contourf(x_mesh, y_mesh, z, 150)
pyplot.scatter(x_plots, y_plots)
pyplot.show()